In [18]:
import pandas as pd 
import numpy as np 
from decimal import Decimal
from tqdm import trange

### Fill 0 to nan function

In [19]:
def fill0_select(select = list,fc=None,kd=None):
    for feature in select:
        kd[feature] =kd[feature].fillna(0)
        fc[feature] = fc[feature].fillna(0)
    return(fc,kd)
def count_na(select = list,fc=None,kd = None):
    fc_nan = {}
    kd_nan = {}
    for feature in select:
        fc_nan[feature] = fc[feature].isna().sum()
        kd_nan[feature] = kd[feature].isna().sum()
    return (fc_nan,kd_nan)


### z score table for HB

In [20]:
HB_z = { 
    (Decimal(0.0),Decimal(0.26666667)):[15.51,2.01],   #1
    (Decimal(0.26666667),Decimal(1.0)):[13.23,2.08], #2
    (Decimal(1.0),Decimal(2.0)):[10.49,1.17],        #3
    (Decimal(2.0),Decimal(4.0)):[10.88,0.94],        #4
    (Decimal(4.0),Decimal(6.0)):[11.77,0.91],        #5
    (Decimal(6.0),Decimal(12.0)):[11.87,0.97],       #6
    (Decimal(12.0),Decimal(24.0)):[12.21,0.95],      #7
    (Decimal(24.0),Decimal(48.0)):[12.51,0.88],      #8
    (Decimal(48.0),Decimal(84.0)):[12.67,0.92],      #9
    (Decimal(84.0),Decimal(120.0)):[13.09,0.97],     #10
    (Decimal(120.0),Decimal(132.0)):[13.31,1.01],    #11
    (Decimal(132.0),Decimal(144.0)):[13.39,1.03],    #12
    (Decimal(144.0),Decimal(216.0)):[13.69,1.47],    #13
}

value = None
for key in HB_z:
    if Decimal(0.2) > key[0] and Decimal(0.2) <= key[1]:
        value = HB_z[key]
        break
print(value)  # output: [15.5, 2]

[15.51, 2.01]


### z score table for eos

In [21]:
eos_z = { 
    (Decimal(0.0),Decimal(0.26666667)):[2.6,2.21], #1
    (Decimal(0.26666667),Decimal(2.0)):[3.24,2.83],#2
    (Decimal(2.0),Decimal(4.0)):[1.99,1.96],       #3
    (Decimal(4.0),Decimal(6.0)):[1.53,1.97],       #4
    (Decimal(6.0),Decimal(12.0)):[0.97,1.54],      #5
    (Decimal(12.0),Decimal(48.0)):[0.87,1.42],     #6
    (Decimal(48.0),Decimal(84.0)):[0.99,1.51],     #7
    (Decimal(84.0),Decimal(120.0)):[1.13,1.79],    #8
    (Decimal(120.0),Decimal(216.0)):[1.26,1.75],   #9
}

value = None
for key in eos_z:
    if Decimal(0.2) > key[0] and Decimal(0.2) <= key[1]:
        value = eos_z[key]
        break
print(value)  # output: [15.5, 2]

[2.6, 2.21]


### z score table for plt 

In [22]:
plt_z = { 
    (Decimal(0.0),Decimal(0.26666667)):[307.3,85.54],   #1
    (Decimal(0.26666667),Decimal(1.0)):[411.55,127.31], #2
    (Decimal(1.0),Decimal(2.0)):[397.72,111.85],        #3
    (Decimal(2.0),Decimal(4.0)):[382.89,107.75],        #4
    (Decimal(4.0),Decimal(6.0)):[323.77,105.32],        #5
    (Decimal(6.0),Decimal(12.0)):[256.98,101.39],       #6
    (Decimal(12.0),Decimal(84.0)):[242.13,83.4],        #7
    (Decimal(84.0),Decimal(216.0)):[231.15,73.49],      #8
}

value = None
for key in plt_z:
    if Decimal(0.26) > key[0] and Decimal(0.26) <= key[1]:
        value = plt_z[key]
        break
print(value)  # output: [15.5, 2]

[307.3, 85.54]


### import data and fill nan element in specific column to 0

In [23]:
fc = pd.read_excel('/home/cosbi/KDGPT/data_processing/fever-CBC,WBC data(20230407).xlsx')
kd = pd.read_excel('/home/cosbi/KDGPT/data_processing/KD-CBC,WBC data(20230407).xlsx')

In [24]:
select=['eosinophil count ','basophil count ','monocyte count ','eosinophil (%)','basophil (%)','monocyte (%)']
fc_nan,kd_nan = count_na(select=select,fc=fc,kd=kd)
print('fc:',f'{fc_nan}\n','kd:',f'{kd_nan}\n')
fc ,kd = fill0_select(select=select,fc=fc,kd=kd)

fc: {'eosinophil count ': 69, 'basophil count ': 41, 'monocyte count ': 35, 'eosinophil (%)': 69, 'basophil (%)': 41, 'monocyte (%)': 35}
 kd: {'eosinophil count ': 217, 'basophil count ': 812, 'monocyte count ': 11, 'eosinophil (%)': 216, 'basophil (%)': 811, 'monocyte (%)': 10}



In [25]:
data_par = ['Hematocrit','Platelets','AST/GOT','ALT/GPT','CRP']
for parameter in data_par:
    fc = fc.astype({'{}'.format(parameter):'str'})#為了下一步，要先將所有資料轉成str
    fc['{}'.format(parameter)] = fc['{}'.format(parameter)].str.extract('(-*\d+\.*\d*)',expand = False)#內部的非數值str都remove
    fc = fc.astype({'{}'.format(parameter):'float'})#移除掉非數值str後將其轉回float
for parameter in data_par:
    kd = kd.astype({'{}'.format(parameter):'str'})#為了下一步，要先將所有資料轉成str
    kd['{}'.format(parameter)] = kd['{}'.format(parameter)].str.extract('(-*\d+\.*\d*)',expand = False)#內部的非數值str都remove
    kd = kd.astype({'{}'.format(parameter):'float'})#移除掉非數值str後將其轉回float

#### count data with more than 18 years and smaller than 0

In [26]:
print('fc origin len:',len(fc),'fc more than 18 years:',len(fc[((fc['age (month)'] > 216.0))]),'fc smaller or equal than 0 days:',len(fc[(fc['age (month)'] == 0.0)]))
print('kd origin len:',len(kd),'kd more than 18 years:',len(kd[((kd['age (month)'] > 216.0))]),'kd smaller or equal than 0 days:',len(kd[(kd['age (month)'] == 0.0)]))

fc origin len: 102801 fc more than 18 years: 75 fc smaller or equal than 0 days: 0
kd origin len: 2904 kd more than 18 years: 0 kd smaller or equal than 0 days: 1


##### filter only the year smaller than 18 years that we keep

In [27]:
fc = fc[((fc['age (month)'] <= 216.0) & (fc['age (month)'] > 0.0))]
kd = kd[((kd['age (month)'] <= 216.0) & (kd['age (month)'] > 0.0))]
fc.reset_index(drop=True,inplace=True)
kd.reset_index(drop=True,inplace=True)
print('fc smaller than 18 years and no 0 day:',len(fc))
print('kd smaller than 18 years and no 0 day:',len(kd))

fc smaller than 18 years and no 0 day: 102726
kd smaller than 18 years and no 0 day: 2903


In [28]:
kd_check_wrong_0 = kd[(kd['eosinophil count '] == 0) & (kd['eosinophil (%)'] != 0)]
kd_check_wrong_0 #有一個但應該不會影響到後續（因為有缺值，後續會被做處理掉）

,ID,性別,age (day),age (month),age (year),WBC (1000),RBC,Hemoglobin,Hematocrit,MCV,...,monocyte count,eosinophil count,basophil count,Band count,AST/GOT,ALT/GPT,CRP,HB \nz score,EOS \nz score,PLT \nz score
1387,7525A304F696C1D0BE9F1AD14CE8BBB8D3B4B664,1,535,17.833333,1.486111,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,35.0,15.0,0.7,-12.922742,0.939976,-2.903188


In [29]:
fc_check_wrong_0 = fc[(fc['eosinophil count '] == 0) & (fc['eosinophil (%)'] != 0)]
fc_check_wrong_0

,ID,性別,age (day),age (month),age (year),WBC (1000),RBC,Hemoglobin,Hematocrit,MCV,...,monocyte count,eosinophil count,basophil count,Band count,AST/GOT,ALT/GPT,CRP,HB z score,EOS z score,PLT z score


In [30]:
fc['count_HB_z'] = ''
fc['count_eos_z'] = ''
fc['count_plt_z'] = ''
kd['count_HB_z'] = ''
kd['count_eos_z'] = ''
kd['count_plt_z'] = ''

##### counting z score by given table

In [31]:
for i in trange(len(fc)):
    hb_mean_std = None
    for key in HB_z:
        if Decimal(fc['age (month)'][i]) > key[0] and Decimal(fc['age (month)'][i]) <= key[1]:
            hb_mean_std = HB_z[key]
            break
    
    eos_z_mean_std = None
    for key in eos_z:
        if Decimal(fc['age (month)'][i]) > key[0] and Decimal(fc['age (month)'][i]) <= key[1]:
            eos_z_mean_std = eos_z[key]
            break

    plt_z_mean_std = None
    for key in plt_z:
        if Decimal(fc['age (month)'][i]) > key[0] and Decimal(fc['age (month)'][i]) <= key[1]:
            plt_z_mean_std = plt_z[key]
            break
        
    try:
        fc['count_HB_z'][i] = (float(fc['Hemoglobin'][i]) -hb_mean_std[0])/hb_mean_std[1]
        fc['count_eos_z'][i] = (float(fc['eosinophil (%)'][i]) - eos_z_mean_std[0])/eos_z_mean_std[1]
        fc['count_plt_z'][i] = (float(fc['Platelets'][i]) - plt_z_mean_std[0])/plt_z_mean_std[1]
    except:
        continue

for i in trange(len(kd)):
    hb_mean_std = None
    for key in HB_z:
        if Decimal(kd['age (month)'][i]) > key[0] and Decimal(kd['age (month)'][i]) <= key[1]:
            hb_mean_std = HB_z[key]
            break
    
    eos_z_mean_std = None
    for key in eos_z:
        if Decimal(kd['age (month)'][i]) > key[0] and Decimal(kd['age (month)'][i]) <= key[1]:
            eos_z_mean_std = eos_z[key]
            break

    plt_z_mean_std = None
    for key in plt_z:
        if Decimal(kd['age (month)'][i]) > key[0] and Decimal(kd['age (month)'][i]) <= key[1]:
            plt_z_mean_std = plt_z[key]
            break
        
    try:
        kd['count_HB_z'][i] = (float(kd['Hemoglobin'][i]) -hb_mean_std[0])/hb_mean_std[1]
        kd['count_eos_z'][i] = (float(kd['eosinophil (%)'][i]) - eos_z_mean_std[0])/eos_z_mean_std[1]
        kd['count_plt_z'][i] = (float(kd['Platelets'][i]) - plt_z_mean_std[0])/plt_z_mean_std[1]
    except:
        continue

  0%|          | 0/102726 [00:00<?, ?it/s]/tmp/ipykernel_3535280/2549473968.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc['count_HB_z'][i] = (float(fc['Hemoglobin'][i]) -hb_mean_std[0])/hb_mean_std[1]
/tmp/ipykernel_3535280/2549473968.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc['count_eos_z'][i] = (float(fc['eosinophil (%)'][i]) - eos_z_mean_std[0])/eos_z_mean_std[1]
/tmp/ipykernel_3535280/2549473968.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [32]:
print(kd.columns)

Index(['ID', '性別', 'age (day)', 'age (month)', 'age (year)', 'WBC (1000)',
       'RBC', 'Hemoglobin', 'Hematocrit', 'MCV', 'MCH', 'MCHC', 'Platelets',
       'Seg (%)', 'Lym (%)', 'monocyte (%)', 'eosinophil (%)', 'basophil (%)',
       'Band (%)', 'Seg count ', 'Lym count ', 'monocyte count ',
       'eosinophil count ', 'basophil count ', 'Band count ', 'AST/GOT',
       'ALT/GPT', 'CRP', 'HB \nz score', 'EOS \nz score', 'PLT \nz score',
       'count_HB_z', 'count_eos_z', 'count_plt_z'],
      dtype='object')


#### count z score diff and output s csv

In [33]:
fc.rename(columns={'HB \nz score':'HB z score','EOS \nz score':'EOS z score','PLT \nz score':'PLT z score'},inplace = True)
fc['hb_diff'] = pd.to_numeric(fc['count_HB_z']) - fc['HB z score']
fc['eos_diff'] = pd.to_numeric(fc['count_eos_z']) - fc['EOS z score']
fc['plt_diff'] = pd.to_numeric(fc['count_plt_z']) - fc['PLT z score']

kd.rename(columns={'HB \nz score':'HB z score','EOS \nz score':'EOS z score','PLT \nz score':'PLT z score'},inplace = True)
kd['hb_diff'] = pd.to_numeric(kd['count_HB_z']) - kd['HB z score']
kd['eos_diff'] = pd.to_numeric(kd['count_eos_z']) - kd['EOS z score']
kd['plt_diff'] = pd.to_numeric(kd['count_plt_z']) - kd['PLT z score']
fc.to_csv('zscore_table_ver2_output/fc_z_score_ver2.csv',index=False)
kd.to_csv('zscore_table_ver2_output/kd_z_score_ver2.csv',index=False)

### import data after counting z score by own

In [34]:
fc_diff = pd.read_csv("zscore_table_ver2_output/fc_z_score_ver2.csv")
kd_diff = pd.read_csv("zscore_table_ver2_output/kd_z_score_ver2.csv")

In [35]:
kd_diff

,ID,性別,age (day),age (month),age (year),WBC (1000),RBC,Hemoglobin,Hematocrit,MCV,...,CRP,HB z score,EOS z score,PLT z score,count_HB_z,count_eos_z,count_plt_z,hb_diff,eos_diff,plt_diff
0,7058,0,254,8.466667,0.705556,18.54,NaN,10.8,NaN,NaN,...,96.70,-1.098901,1.055392,0.246737,-1.103093,1.058442,0.246770,-0.004192,0.003050,0.000033
1,7059,0,73,2.433333,0.202778,7.95,NaN,10.9,NaN,NaN,...,54.60,0.000000,-0.453247,-0.463019,0.021277,-0.454082,-0.463016,0.021277,-0.000834,0.000002
2,7060,1,1530,51.000000,4.250000,16.10,NaN,11.0,NaN,NaN,...,94.08,-1.855490,-0.393267,-0.505175,-1.815217,-0.390728,-0.505156,0.040273,0.002539,0.000019
3,7061,1,1530,51.000000,4.250000,15.71,NaN,11.5,NaN,NaN,...,53.71,-1.309758,0.071224,0.466021,-1.271739,0.072848,0.466067,0.038019,0.001624,0.000046
4,7062,1,178,5.933333,0.494444,15.26,NaN,9.5,NaN,NaN,...,71.95,-2.515091,-0.727017,0.695283,-2.494505,-0.725888,0.695310,0.020585,0.001128,0.000026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2898,FF9D5C6850B3DE8B34508D7E08BF63BCE54092AA,1,338,11.266667,0.938889,27.00,4.55,11.1,34.5,75.8,...,236.00,-0.792055,2.609617,0.049473,-0.793814,2.616883,0.049512,-0.001760,0.007266,0.000039
2899,FFCE1F940E4D9751E3C3FBDC8E3F88E80E40206C,0,1539,51.300000,4.275000,8.70,4.42,11.9,35.5,80.3,...,57.70,-0.842065,1.265629,-0.181443,-0.836957,1.264901,-0.181415,0.005109,-0.000728,0.000028
2900,FFD3B41E1902709D0DDC9AD74976D620C611615D,0,372,12.400000,1.033333,3.70,4.07,11.4,34.7,85.3,...,0.90,-0.861036,0.939976,-0.960797,-0.852632,0.936620,-0.960791,0.008405,-0.003356,0.000006
2901,FFD8FFAFD64762D9D512629CC49BB855F0385AE5,1,570,19.000000,1.583333,16.00,4.50,12.0,34.7,77.1,...,26.97,-0.226210,1.503809,1.700998,-0.221053,1.500000,1.701079,0.005157,-0.003809,0.000081


In [36]:
fc_diff

,ID,性別,age (day),age (month),age (year),WBC (1000),RBC,Hemoglobin,Hematocrit,MCV,...,CRP,HB z score,EOS z score,PLT z score,count_HB_z,count_eos_z,count_plt_z,hb_diff,eos_diff,plt_diff
0,F5F3EF927AF17059A99690AD8038731C27D567BB,1.0,1,0.033333,0.002778,24.6,6.08,21.7,62.5,102.8,...,0.80,3.073075,-0.043649,-1.230974,3.079602,-0.045249,-1.231003,0.006527,-0.001600,-0.000029
1,CBB0CDC612C998917AD06556E2444CCC719894A1,1.0,1,0.033333,0.002778,22.1,4.39,15.6,44.9,102.3,...,43.20,0.042968,-0.950340,-0.202251,0.044776,-0.950226,-0.202245,0.001808,0.000114,0.000006
2,27098D07E75F404957EF0571B1604FCD9B50B2F6,1.0,1,0.033333,0.002778,16.5,6.09,19.8,55.5,91.1,...,9.70,2.129271,-0.496995,-0.821823,2.134328,-0.497738,-0.821838,0.005057,-0.000743,-0.000015
3,CCD85E5A1EC56F50CD3CE382FC24B166CDCB379E,1.0,1,0.033333,0.002778,13.3,3.65,13.7,37.9,103.8,...,4.20,-0.900836,-0.270322,-0.716612,-0.900498,-0.271493,-0.716624,0.000338,-0.001171,-0.000011
4,861121B9FEB3B7E09BC58FCFF51B75358E068799,1.0,2,0.066667,0.005556,13.1,4.13,13.7,39.1,94.7,...,0.10,-0.900836,-0.270322,-1.090694,-0.900498,-0.271493,-1.090718,0.000338,-0.001171,-0.000024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102721,94B248D7B5209A9A09B7096716C0CEC1D785EB7B,1.0,6475,215.833333,17.986111,9.9,5.27,14.0,41.4,78.6,...,76.80,0.212857,-0.722202,0.025121,0.210884,-0.720000,0.025173,-0.001973,0.002202,0.000052
102722,30A8766FAC9E77CEB35EAC3866C97E49CD680C52,0.0,6476,215.866667,17.988889,10.4,4.50,13.3,38.2,84.9,...,136.00,-0.264707,-0.664945,-1.281207,-0.265306,-0.662857,-1.281127,-0.000599,0.002088,0.000080
102723,6DB265F9C6FE51DA57DFA5D231EB843D3FE29C99,0.0,6477,215.900000,17.991667,13.1,4.28,13.1,38.0,88.8,...,47.00,-0.401154,-0.435917,-0.491967,-0.401361,-0.434286,-0.491904,-0.000206,0.001632,0.000063
102724,07BB795CEF6AF675AB65284D5C758776B09AE6C7,0.0,6477,215.900000,17.991667,9.8,5.06,9.9,32.1,63.4,...,33.18,-2.584307,-0.722202,0.896006,-2.578231,-0.720000,0.896040,0.006076,0.002202,0.000034


In [37]:
print('FC:\nmax hb diff:',max(abs(fc_diff['hb_diff'])),'\nmax eos diff',max(abs(fc_diff['eos_diff'])),'\nmax plt diff',max(abs(fc_diff['plt_diff'])))
print('KD:\nmax hb diff:',max(abs(kd_diff['hb_diff'])),'\nmax eos diff',max(abs(kd_diff['eos_diff'])),'\nmax plt diff',max(abs(kd_diff['plt_diff'])))

FC:
max hb diff: 0.0484854020659639 
max eos diff 0.0353717644060083 
max plt diff 0.000334609858525
KD:
max hb diff: 0.6597230349984708 
max eos diff 0.0353717644059976 
max plt diff 0.0002819224946755


##### output HB z score diff over threshold 

In [38]:
fc_diff_HB_over = fc_diff[(abs((fc_diff['hb_diff'])>0.1) | (fc_diff['hb_diff']<-0.1))]
kd_diff_HB_over = kd_diff[(abs((kd_diff['hb_diff'])>0.1) | (kd_diff['hb_diff']<-0.1))]

In [39]:
fc_diff_HB_over.sort_values(['age (month)'])

,ID,性別,age (day),age (month),age (year),WBC (1000),RBC,Hemoglobin,Hematocrit,MCV,...,CRP,HB z score,EOS z score,PLT z score,count_HB_z,count_eos_z,count_plt_z,hb_diff,eos_diff,plt_diff


In [40]:
kd_diff_HB_over.sort_values(['age (month)'])

,ID,性別,age (day),age (month),age (year),WBC (1000),RBC,Hemoglobin,Hematocrit,MCV,...,CRP,HB z score,EOS z score,PLT z score,count_HB_z,count_eos_z,count_plt_z,hb_diff,eos_diff,plt_diff
1967,AA82D3D2A56CDAAF50D1D6ADA1633FA8BCDE50DC,1,4215,140.500000,11.708333,13.1,3.67,11.5,32.8,89.4,...,88.10,-1.492731,-0.493174,0.474171,-1.834951,-0.491429,0.474214,-0.342221,0.001746,0.000043
1674,8F5C4D321E9DC9368D85B9C481C3A2942BDD80AE,0,4274,142.466667,11.872222,17.8,3.77,10.4,31.3,83.0,...,77.63,-2.243190,2.140642,1.004867,-2.902913,2.137143,1.004899,-0.659723,-0.003499,0.000032


In [41]:
fc_diff_HB_over.to_csv('zscore_table_ver2_output/fc diff HB over 0.1_ver2.csv',index=False)
kd_diff_HB_over.to_csv('zscore_table_ver2_output/kd diff HB over 0.1_ver2.csv',index=False)

##### output EOS z score diff over threshold 

In [42]:
fc_diff_eos_over = fc_diff[(abs((fc_diff['eos_diff'])>0.1) | (fc_diff['eos_diff']<-0.1))]
kd_diff_eos_over = kd_diff[(abs((kd_diff['eos_diff'])>0.1) | (kd_diff['eos_diff']<-0.1))]

In [43]:
fc_diff_eos_over.sort_values(['age (month)'])

,ID,性別,age (day),age (month),age (year),WBC (1000),RBC,Hemoglobin,Hematocrit,MCV,...,CRP,HB z score,EOS z score,PLT z score,count_HB_z,count_eos_z,count_plt_z,hb_diff,eos_diff,plt_diff


In [44]:
kd_diff_eos_over.sort_values(['age (month)'])

,ID,性別,age (day),age (month),age (year),WBC (1000),RBC,Hemoglobin,Hematocrit,MCV,...,CRP,HB z score,EOS z score,PLT z score,count_HB_z,count_eos_z,count_plt_z,hb_diff,eos_diff,plt_diff


In [45]:
fc_diff_eos_over.to_csv('zscore_table_ver2_output/fc diff EOS over 0.1_ver2.csv',index=False)
kd_diff_eos_over.to_csv('zscore_table_ver2_output/kd diff EOS over 0.1_ver2.csv',index=False)

##### output PLT z score diff over threshold 

In [46]:
fc_diff_plt_over = fc_diff[(abs((fc_diff['plt_diff'])>0.1) | (fc_diff['plt_diff']<-0.1))]
kd_diff_plt_over = kd_diff[(abs((kd_diff['plt_diff'])>0.1) | (kd_diff['plt_diff']<-0.1))]

In [47]:
fc_diff_plt_over.to_csv('zscore_table_ver2_output/fc diff PLT over 0.1_ver2.csv',index=False)
kd_diff_plt_over.to_csv('zscore_table_ver2_output/kd diff PLT over 0.1_ver2.csv',index=False)

### import data after counting z score

In [48]:
fc_diff = pd.read_csv("zscore_table_ver2_output/fc_z_score_ver2.csv")
kd_diff = pd.read_csv("zscore_table_ver2_output/kd_z_score_ver2.csv")

##### output data that replace the Z score column that get from source by own counting, and remove non necessary column(diff, count)

In [ ]:
fc_diff['HB z score'] = fc_diff['count_HB_z']
fc_diff['EOS z score'] = fc_diff['count_eos_z']
fc_diff['PLT z score'] = fc_diff['count_plt_z']

fc_diff.drop(columns=['hb_diff','eos_diff','plt_diff','count_HB_z','count_eos_z','count_plt_z'],inplace=True)


kd_diff['HB z score'] = kd_diff['count_HB_z']
kd_diff['EOS z score'] = kd_diff['count_eos_z']
kd_diff['PLT z score'] = kd_diff['count_plt_z']
kd_diff.drop(columns=['hb_diff','eos_diff','plt_diff','count_HB_z','count_eos_z','count_plt_z'],inplace=True)

fc_diff.to_csv('zscore_table_ver2_output/fc_ver2.csv',index=False)
kd_diff.to_csv('zscore_table_ver2_output/kd_ver2.csv',index=False)